#Using deep features to build an image classifier

#Fire up GraphLab Create

In [16]:
import turicreate as tc

#Load a common image analysis dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set.  

In [17]:
#(poner atencion de que ya esta partido)
image_train = tc.SFrame('datasets/image_train_data/')
image_test = tc.SFrame('datasets/image_test_data/')

#Exploring the image data

In [18]:
#tc.canvas.set_target('ipynb')

In [19]:
#image_train['image'].show()
#aqui se muestran las imagenes de la columna (imagenes)

#Train a classifier on the raw image pixels

We first start by training a classifier on just the raw pixels of the image.

In [20]:
#funcion logistica para clasificar, en target es la columna
raw_pixel_model = tc.logistic_classifier.create(image_train,target='label',
                                              features=['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 1893

...

| 6         | 14       | 1.000000  | 1.995653     | 0.439514          | 0.383929            |

| 10        | 19       | 1.000000  | 2.834454     | 0.503962          | 0.464286            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

##Make a prediction with the simple model based on raw pixels

In [21]:
image_test[0:3]['image'].explore()
#los primeros elementos

Materializing SFrame...

Done.

In [ ]:
image_test[0:3]['label']

In [ ]:
raw_pixel_model.predict(image_test[0:3])
#ver que el predict se 'come' los datos del test
#se equivoco en las 3 predicciones

The model makes wrong predictions for all three images.

#Evaluating raw pixel model on test data

In [ ]:
raw_pixel_model.evaluate(image_test)
#dado que nos equivocamos vemos lo evaluamos
#el accuracy vemos .048 muy bajo
#esto dado que clasificamos con el label de las imagenes

The accuracy of this model is poor, getting only about 46% accuracy.

#Can we improve the model using deep features

We only have 2005 data points, so it is not possible to train a deep neural network effectively with so little data.  Instead, we will use transfer learning: using deep features trained on the full ImageNet dataset, we will train a simple model on this small dataset.

In [ ]:
len(image_train)

##Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [ ]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

As we can see, the column deep_features already contains the pre-computed deep features for this data. 

In [ ]:
#extrae las columnas que necesitamos para entrenar
image_train.head()

#Given the deep features, let's train a classifier

In [ ]:
#ahora ajustamos a los deep features
deep_features_model = tc.logistic_classifier.create(image_train,
                                                         features=['deep_features'],
                                                         target='label')

##Apply the deep features model to first few images of test set

In [ ]:
image_test[0:3]['image'].show()
##recordar que compara los verdaderos vs loas predicciones

In [ ]:
deep_features_model.predict(image_test[0:3])

The classifier with deep features gets all of these images right!

#Compute test_data accuracy of deep_features_model

As we can see, deep features provide us with significantly better accuracy (about 78%)

In [ ]:
deep_features_model.evaluate(image_test)
#vemos que mejoró el accuracy

In [ ]:
#imagenet challlenge this data taken